# Lab 03: Regression Fundamentals

**ING3513 - Introduction to Artificial Intelligence and Machine Learning**

In this lab, you'll learn the core concepts of regression — fitting curves to data and understanding when those fits can go wrong.

**What you'll learn:**

- Polynomial fitting with NumPy
- Mean Squared Error (MSE) as a loss function
- Train/test splits for honest evaluation
- Overfitting — why "better" training performance can be worse
- The importance of skepticism in ML results


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# Configure plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

print("Libraries loaded successfully!")

## Background: Polynomials and Polynomial Fitting

### What is a polynomial?

A **polynomial** is a mathematical expression with terms of the form $ax^n$, where $a$ is a coefficient and $n$ is a non-negative integer (the degree of that term).

Examples:

- **Degree 1 (linear):** $y = 2x + 1$ — a straight line
- **Degree 2 (quadratic):** $y = x^2 - 2x + 1$ — a parabola
- **Degree 3 (cubic):** $y = x^3 - 3x$ — an S-shaped curve

The **degree** of a polynomial is the highest power of $x$. Higher-degree polynomials can represent more complex, "wiggly" curves.

### What is polynomial fitting?

**Polynomial fitting** (or polynomial regression) finds the polynomial that best matches a set of data points. Given data, we choose a degree $d$ and find coefficients $a_0, a_1, \ldots, a_d$ such that:

$$y = a_0 + a_1 x + a_2 x^2 + \cdots + a_d x^d$$

The "best" fit minimizes the total error between the polynomial's predictions and the actual data points.

### Why does this matter for ML?

Polynomial fitting is a simple example of **model selection** — choosing the right complexity for your model. It illustrates a fundamental tension in machine learning:

- Too simple (low degree) → **underfitting** — can't capture the pattern
- Too complex (high degree) → **overfitting** — captures noise instead of pattern

This lab uses polynomials to build intuition for these concepts, which apply to ALL machine learning models.


## 1. The Scenario — Fitting Curves to Data

Imagine you've collected measurements and want to discover the underlying pattern. This is the essence of regression: given noisy data, find a function that describes it.

Let's start with a simple example — data generated from a quadratic function with some noise.


In [ ]:
# Generate sample data from a quadratic function: y = x² - 2x + 1 = (x-1)²
np.random.seed(42)

X = np.linspace(-3, 3, 40).reshape(
    -1, 1
)  # sklearn requires 2D: (n_samples, n_features)
y_true = X.flatten() ** 2 - 2 * X.flatten() + 1  # True underlying function
y = y_true + np.random.normal(0, 1.0, size=y_true.shape)  # Add noise (σ=1.0)

# Visualize the data
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X.flatten(), y=y, s=80, alpha=0.7, label="Observed data")
sns.lineplot(
    x=X.flatten(),
    y=y_true,
    color="green",
    linestyle="--",
    label="True function: $(x-1)^2$",
)
plt.xlabel("X")
plt.ylabel("y")
plt.title("Our Data: Quadratic Function + Noise (σ = 1.0)")
plt.legend()
plt.show()

print(f"Number of data points: {len(X)}")
print(f"X shape: {X.shape} — Note: sklearn needs 2D arrays (samples × features)")

## 2. Polynomial Fitting with sklearn

sklearn uses a **Pipeline** that combines:

1. **`PolynomialFeatures`** — transforms X into [1, X, X², X³, ...]
2. **`LinearRegression`** — fits coefficients to minimize squared error

This pattern (`fit()` then `predict()`) is the same for ALL sklearn models!


In [ ]:
# Create a polynomial regression model (degree 2)
model = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False), LinearRegression()
)

# Fit the model to our data
model.fit(X, y)

# Get the coefficients
poly_features = model.named_steps["polynomialfeatures"]
lin_reg = model.named_steps["linearregression"]
print(f"Intercept: {lin_reg.intercept_:.4f}")
print(f"Coefficients: {lin_reg.coef_}")
print("True coefficients: intercept=1, coef=[-2, 1] for (x-1)²")

# Generate predictions for smooth plotting
X_smooth = np.linspace(-3, 3, 200).reshape(-1, 1)
y_fitted = model.predict(X_smooth)
y_true_smooth = X_smooth.flatten() ** 2 - 2 * X_smooth.flatten() + 1

# Plot the result
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X.flatten(), y=y, s=80, alpha=0.7, label="Observed data")
sns.lineplot(
    x=X_smooth.flatten(),
    y=y_true_smooth,
    color="green",
    linestyle="--",
    linewidth=2,
    label="True function: $(x-1)^2$",
)
sns.lineplot(
    x=X_smooth.flatten(),
    y=y_fitted,
    color="red",
    linewidth=2,
    label="Fitted polynomial (deg 2)",
)
plt.xlabel("X")
plt.ylabel("y")
plt.title("Polynomial Fit to Data")
plt.legend()
plt.show()

## 3. Mean Squared Error (MSE)

How do we measure how good our fit is? The **Mean Squared Error** is the average of squared differences between predicted and actual values:

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

where $y_i$ is the actual value and $\hat{y}_i$ is the predicted value.

**Why squared?**

- Penalizes large errors more than small ones
- Always positive (no cancellation of positive/negative errors)
- Mathematically convenient for optimization


In [ ]:
# Compute MSE using sklearn's function
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(
    f"\nInterpretation: On average, our predictions are off by √{mse:.4f} ≈ {np.sqrt(mse):.2f}"
)

# Note: sklearn also provides R² score (coefficient of determination)
r2 = model.score(X, y)  # Built-in method returns R²
print(f"R² score: {r2:.4f} (1.0 = perfect fit, 0.0 = predicts mean)")

## 4. The Problem — What If We Fit the Wrong Degree?

What happens if we use the wrong polynomial degree? Let's try:

- **Degree 1** (linear) — too simple?
- **Degree 2** (quadratic) — just right?
- **Degree 9** (high-order) — too complex?


In [ ]:
# Fit polynomials of different degrees
degrees = [1, 2, 9]
models = {}

for deg in degrees:
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False), LinearRegression()
    )
    model.fit(X, y)
    models[deg] = model

# Visualize all three fits
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
titles = ["Degree 1 (Underfit?)", "Degree 2 (Good Fit?)", "Degree 9 (Overfit?)"]
y_true_smooth = X_smooth.flatten() ** 2 - 2 * X_smooth.flatten() + 1

for ax, deg, title in zip(axes, degrees, titles):
    y_pred = models[deg].predict(X_smooth)
    mse = mean_squared_error(y, models[deg].predict(X))

    sns.scatterplot(x=X.flatten(), y=y, s=60, alpha=0.7, ax=ax)
    ax.plot(X_smooth.flatten(), y_true_smooth, "g--", alpha=0.6, label="True function")
    sns.lineplot(
        x=X_smooth.flatten(),
        y=y_pred,
        color="red",
        linewidth=2,
        ax=ax,
        label=f"Fit (deg {deg})",
    )
    ax.set_title(f"{title}\nMSE = {mse:.4f}")
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    ax.set_ylim(-2, 18)
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

# Print MSE comparison
print("MSE Comparison:")
for deg in degrees:
    mse = mean_squared_error(y, models[deg].predict(X))
    print(f"  Degree {deg}: {mse:.4f}")

### Wait... Degree 9 has the _lowest_ MSE!

The degree-9 polynomial fits our data points better than the true degree-2 polynomial. Does that mean we should use degree 9?

**No!** This is a critical lesson: the degree-9 polynomial is fitting the _noise_, not the underlying pattern. It will perform terribly on new data.

But how can we detect this problem?


## 5. Train/Test Split — The Solution

The solution is to **hide some data** during training. We only use part of our data to fit the model, then evaluate on the hidden portion.

- **Training set**: Used to fit the polynomial
- **Test set**: Hidden during fitting, used for honest evaluation

If a model has low training error but high test error, it's **overfitting** — memorizing the training data rather than learning the pattern.


In [ ]:
# Split data: 70% training, 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Visualize the split
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x=X_train.flatten(), y=y_train, s=80, label=f"Training data ({len(X_train)} points)"
)
sns.scatterplot(
    x=X_test.flatten(),
    y=y_test,
    s=80,
    marker="s",
    label=f"Test data ({len(X_test)} points)",
)
plt.xlabel("X")
plt.ylabel("y")
plt.title("Train/Test Split")
plt.legend()
plt.show()

In [ ]:
# Now fit using ONLY training data and evaluate on BOTH sets
degrees = [1, 2, 9]

print("Degree | Train MSE | Test MSE")
print("-" * 35)

for deg in degrees:
    # Create and fit model on training data only
    model = make_pipeline(
        PolynomialFeatures(degree=deg, include_bias=False), LinearRegression()
    )
    model.fit(X_train, y_train)

    # Compute MSE on both sets
    train_mse = mean_squared_error(y_train, model.predict(X_train))
    test_mse = mean_squared_error(y_test, model.predict(X_test))

    print(f"  {deg:4d}  |  {train_mse:7.4f}  |  {test_mse:7.4f}")

### 💡 Now the truth is revealed!

Degree 9 has the lowest _training_ MSE, but look at its _test_ MSE — it's worse than degree 2!

This is **overfitting**: the model fits the training data too well, including the noise, so it fails to generalize to new data. Notice also the _gap_ between train and test MSE for degree 9 — that gap is the telltale sign.


## 6. Helper Functions for Experiments

Before we dive into larger experiments, let's define some helper functions to generate data and run fits systematically.


In [ ]:
# Define our "true" polynomial coefficients (for data generation only)
# Format: [coef for x^n, ..., coef for x^1, coef for x^0]
TRUE_POLY_COEFFS = {
    1: [2, 1],  # y = 2x + 1
    2: [1, -2, 1],  # y = x² - 2x + 1 = (x-1)²
    3: [0.5, 0, -2, 0],  # y = 0.5x³ - 2x
}


def generate_polynomial_data(
    degree, n_points=40, noise_std=0.5, x_range=(-3, 3), seed=None
):
    """
    Generate data from a true polynomial with added Gaussian noise.

    Parameters:
    - degree: Which true polynomial to use (1, 2, or 3)
    - n_points: Number of data points
    - noise_std: Standard deviation of Gaussian noise
    - x_range: Tuple of (min_x, max_x)
    - seed: Random seed for reproducibility

    Returns:
    - X: Input values (2D array for sklearn)
    - y: Output values (with noise)
    - y_true: True output values (without noise)
    """
    if seed is not None:
        np.random.seed(seed)

    X = np.linspace(x_range[0], x_range[1], n_points).reshape(-1, 1)
    poly = np.poly1d(TRUE_POLY_COEFFS[degree])
    y_true = poly(X.flatten())
    y = y_true + np.random.normal(0, noise_std, size=y_true.shape)

    return X, y, y_true


def fit_and_evaluate(X_train, y_train, X_test, y_test, fit_degree):
    """
    Fit a polynomial model and return train/test MSE.

    Returns:
    - model: The fitted sklearn pipeline
    - train_mse: MSE on training data
    - test_mse: MSE on test data
    """
    model = make_pipeline(
        PolynomialFeatures(degree=fit_degree, include_bias=False), LinearRegression()
    )
    model.fit(X_train, y_train)

    train_mse = mean_squared_error(y_train, model.predict(X_train))
    test_mse = mean_squared_error(y_test, model.predict(X_test))

    return model, train_mse, test_mse


print("Helper functions defined!")

## 7. Generating Multiple Datasets

Let's create datasets from polynomials of different degrees, each with the same small amount of noise. This lets us see what "ideal" data looks like for each underlying function.


In [ ]:
# Generate datasets from degree 1, 2, and 3 polynomials
data_degrees = [1, 2, 3]
datasets = {}

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, deg in zip(axes, data_degrees):
    X_data, y_data, y_true = generate_polynomial_data(deg, noise_std=1.0, seed=42 + deg)
    datasets[deg] = (X_data, y_data, y_true)

    # Plot
    sns.scatterplot(
        x=X_data.flatten(), y=y_data, s=60, alpha=0.7, ax=ax, label="Noisy data"
    )
    X_smooth = np.linspace(-3, 3, 200)
    y_smooth = np.poly1d(TRUE_POLY_COEFFS[deg])(X_smooth)
    sns.lineplot(
        x=X_smooth,
        y=y_smooth,
        color="green",
        linestyle="--",
        ax=ax,
        label="True function",
    )
    ax.set_title(f"Data from Degree-{deg} Polynomial")
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    ax.legend()

plt.tight_layout()
plt.show()

print("Datasets generated from polynomials of degree 1, 2, and 3")

## 8. The Fitting Matrix

Now let's systematically explore what happens when we fit different polynomial degrees to different datasets.

We'll fit all combinations:

- Data generated from degrees: 1, 2, 3
- Fitting polynomial degrees: 1, 2, 3, 9

This gives us a 3×4 matrix of results. For each combination, we compute both train MSE and test MSE.


In [ ]:
# Configuration
data_degrees = [1, 2, 3]  # Degrees used to generate the data
fit_degrees = [1, 2, 3, 9]  # Degrees we'll try to fit
noise_std = 1.0  # Same as initial data generation

# Initialize result matrices
n_data = len(data_degrees)
n_fits = len(fit_degrees)
train_mse_matrix = np.zeros((n_data, n_fits))
test_mse_matrix = np.zeros((n_data, n_fits))

# Run all combinations
for i, data_deg in enumerate(data_degrees):
    # Generate data for this source polynomial
    X_data, y_data, _ = generate_polynomial_data(
        data_deg, noise_std=noise_std, seed=42 + data_deg
    )

    # Split into train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_data, y_data, test_size=0.3, random_state=42
    )

    for j, fit_deg in enumerate(fit_degrees):
        # Fit and evaluate
        _, train_mse, test_mse = fit_and_evaluate(
            X_train, y_train, X_test, y_test, fit_deg
        )

        train_mse_matrix[i, j] = train_mse
        test_mse_matrix[i, j] = test_mse

print("Fitting complete! Results stored in train_mse_matrix and test_mse_matrix")

In [ ]:
# Visualize results as heatmaps
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training MSE heatmap
sns.heatmap(
    train_mse_matrix,
    annot=True,
    fmt=".3f",
    xticklabels=[f"Fit deg {d}" for d in fit_degrees],
    yticklabels=[f"Data deg {d}" for d in data_degrees],
    cmap="YlOrRd",
    ax=axes[0],
)
axes[0].set_title("Training MSE\n(Lower = Better Fit to Training Data)")
axes[0].set_xlabel("Fitting Polynomial Degree")
axes[0].set_ylabel("Data Generated From Degree")

# Test MSE heatmap
sns.heatmap(
    test_mse_matrix,
    annot=True,
    fmt=".3f",
    xticklabels=[f"Fit deg {d}" for d in fit_degrees],
    yticklabels=[f"Data deg {d}" for d in data_degrees],
    cmap="YlOrRd",
    ax=axes[1],
)
axes[1].set_title("Test MSE\n(Lower = Better Generalization)")
axes[1].set_xlabel("Fitting Polynomial Degree")
axes[1].set_ylabel("Data Generated From Degree")

plt.tight_layout()
plt.show()

### Observations from the Fitting Matrix

**Look at the Training MSE (left):**

- Degree 9 almost always has the lowest training MSE
- Higher-degree polynomials can always fit training data better (or at least as well)

**Look at the Test MSE (right):**

- The best test MSE is typically when fit degree ≈ data degree
- Degree 9 often has _worse_ test MSE despite _better_ training MSE

**Key insight:** Training MSE alone is misleading! We must always evaluate on held-out test data.


## 9. Visualizing Underfit / Good Fit / Overfit

Let's visualize three specific cases to build intuition:

1. **Underfit**: Degree-1 fit on degree-3 data (too simple)
2. **Good fit**: Degree-3 fit on degree-3 data (just right)
3. **Overfit**: Degree-9 fit on degree-3 data (too complex)


In [ ]:
# Generate degree-3 data
X_deg3, y_deg3, _ = generate_polynomial_data(3, noise_std=1.0, seed=45)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(
    X_deg3, y_deg3, test_size=0.3, random_state=42
)

# Fit different degree polynomials
fit_cases = [
    (1, "Underfit", "steelblue"),
    (3, "Good Fit", "green"),
    (9, "Overfit", "red"),
]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
X_smooth = np.linspace(-3, 3, 200).reshape(-1, 1)

for ax, (fit_deg, title, color) in zip(axes, fit_cases):
    model, train_mse, test_mse = fit_and_evaluate(
        X_train_3, y_train_3, X_test_3, y_test_3, fit_deg
    )

    # Plot
    sns.scatterplot(
        x=X_train_3.flatten(), y=y_train_3, s=60, alpha=0.7, ax=ax, label="Train"
    )
    sns.scatterplot(
        x=X_test_3.flatten(),
        y=y_test_3,
        s=60,
        marker="s",
        alpha=0.7,
        ax=ax,
        label="Test",
    )
    ax.plot(
        X_smooth.flatten(),
        model.predict(X_smooth),
        color=color,
        linewidth=2,
        label=f"Fit (deg {fit_deg})",
    )
    ax.plot(
        X_smooth.flatten(),
        np.poly1d(TRUE_POLY_COEFFS[3])(X_smooth.flatten()),
        "k--",
        alpha=0.5,
        label="True function",
    )

    ax.set_title(
        f"{title} (Degree {fit_deg})\nTrain MSE: {train_mse:.3f} | Test MSE: {test_mse:.3f}"
    )
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    ax.legend(fontsize=8)
    ax.set_ylim(-8, 8)

plt.tight_layout()
plt.show()

## 10. The Effect of Noise

What happens as data gets noisier? Let's run our fitting experiment across different noise levels.

**Hypothesis:** With more noise, high-degree polynomials will:

- Have even lower training MSE (they fit noise better!)
- Have much higher test MSE (they generalize worse!)


In [ ]:
# Noise experiment configuration
noise_levels = [0.3, 0.5, 1.0, 2.0, 4.0]
data_deg = 2  # Focus on degree-2 data
fit_degrees = [1, 2, 3, 9]

# Store results for each noise level
results = {noise: {"train": [], "test": []} for noise in noise_levels}

for noise_std in noise_levels:
    # Generate data with this noise level
    X_data, y_data, _ = generate_polynomial_data(data_deg, noise_std=noise_std, seed=42)
    X_train, X_test, y_train, y_test = train_test_split(
        X_data, y_data, test_size=0.3, random_state=42
    )

    for fit_deg in fit_degrees:
        _, train_mse, test_mse = fit_and_evaluate(
            X_train, y_train, X_test, y_test, fit_deg
        )
        results[noise_std]["train"].append(train_mse)
        results[noise_std]["test"].append(test_mse)

print("Noise experiment complete!")

In [ ]:
# Visualize how data looks at different noise levels
fig, axes = plt.subplots(1, 5, figsize=(18, 4))

for ax, noise_std in zip(axes, noise_levels):
    X_data, y_data, y_true = generate_polynomial_data(
        data_deg, noise_std=noise_std, seed=42
    )

    sns.scatterplot(x=X_data.flatten(), y=y_data, s=50, alpha=0.7, ax=ax)
    ax.plot(
        np.linspace(-3, 3, 100),
        np.poly1d(TRUE_POLY_COEFFS[data_deg])(np.linspace(-3, 3, 100)),
        "g--",
        alpha=0.7,
        label="True",
    )
    ax.set_title(f"Noise σ = {noise_std}")
    ax.set_xlabel("X")
    ax.set_ylim(-15, 25)

axes[0].set_ylabel("y")
plt.suptitle("Degree-2 Data with Increasing Noise", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the Train-Test GAP — the clearest overfitting signal
fig, ax = plt.subplots(figsize=(10, 6))

# Prepare data for plotting
gap_data = {deg: [] for deg in fit_degrees}

for noise_std in noise_levels:
    for i, deg in enumerate(fit_degrees):
        train_mse = results[noise_std]["train"][i]
        test_mse = results[noise_std]["test"][i]
        gap_data[deg].append(test_mse - train_mse)

# Plot the gap for each polynomial degree
colors = ["steelblue", "green", "orange", "red"]
for deg, color in zip(fit_degrees, colors):
    ax.plot(
        noise_levels,
        gap_data[deg],
        "o-",
        label=f"Fit deg {deg}",
        linewidth=2,
        markersize=8,
        color=color,
    )

ax.set_xlabel("Noise Level (σ)", fontsize=12)
ax.set_ylabel("Test MSE − Train MSE (Gap)", fontsize=12)
ax.set_title("The Overfitting Gap: How Much Worse is Test vs Train?", fontsize=14)
ax.legend()
ax.axhline(y=0, color="gray", linestyle="--", alpha=0.5, label="No gap (ideal)")

plt.tight_layout()
plt.show()

print("📊 The gap between test and train MSE is the signature of overfitting.")
print("   Degree 9 has the largest gap at every noise level — it overfits the most!")

### Key Observations from the Gap Plot

**The Train-Test Gap reveals overfitting:**

- **Degree 2** (the correct model) has a gap near zero at all noise levels — train and test performance match
- **Degree 9** has the largest gap, and it grows with noise — clear overfitting!
- **Degree 1** (underfit) also has a small gap, but both train AND test are bad

**Why the gap matters:**

In practice, you won't know the "true" function. But you CAN always measure the train-test gap. A large gap is a warning sign that your model is too complex for your data.


## 11. The Overfitting Deep Dive

Let's visualize exactly what happens when we fit high-degree polynomials to noisy data. The overfit model will look "wiggly" as it tries to pass through every noisy data point.


In [ ]:
# Compare degree-2 vs degree-9 fits at high noise
high_noise = 2.0
X_data, y_data, _ = generate_polynomial_data(2, noise_std=high_noise, seed=42)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, random_state=42
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
X_smooth = np.linspace(-3, 3, 200).reshape(-1, 1)

for ax, fit_deg, color, title in zip(
    axes,
    [2, 9],
    ["green", "purple"],
    ["Degree 2 (Correct Model)", "Degree 9 (Overfit)"],
):
    model, train_mse, test_mse = fit_and_evaluate(
        X_train, y_train, X_test, y_test, fit_deg
    )

    sns.scatterplot(
        x=X_train.flatten(), y=y_train, s=80, ax=ax, label="Train", alpha=0.7
    )
    sns.scatterplot(
        x=X_test.flatten(), y=y_test, s=80, marker="s", ax=ax, label="Test", alpha=0.7
    )
    ax.plot(
        X_smooth.flatten(),
        model.predict(X_smooth),
        color=color,
        linewidth=2,
        label=f"Fit (deg {fit_deg})",
    )
    ax.plot(
        X_smooth.flatten(),
        np.poly1d(TRUE_POLY_COEFFS[2])(X_smooth.flatten()),
        "k--",
        alpha=0.5,
        label="True function",
    )

    ax.set_title(f"{title}\nTrain MSE: {train_mse:.2f} | Test MSE: {test_mse:.2f}")
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    ax.legend()
    ax.set_ylim(-15, 25)

plt.suptitle(f"High Noise Data (σ = {high_noise})", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

print("\n🔍 Notice how the degree-9 polynomial 'wiggles' to fit every training point!")
print("   But it completely misses the test points and the true underlying pattern.")

## 12. The Classic Overfitting Curve

This is the most iconic visualization in machine learning: test error forms a **U-shape** as model complexity increases.

- **Left side of U:** Underfitting — model too simple
- **Bottom of U:** Sweet spot — right complexity
- **Right side of U:** Overfitting — model too complex

Compare this to neural network training curves where the X-axis is epochs instead of polynomial degree — same phenomenon!


In [ ]:
# The classic overfitting curve: U-shaped test error
noise_std = 1.5
X_data, y_data, _ = generate_polynomial_data(2, noise_std=noise_std, seed=42)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, random_state=42
)

# Test up to degree 15 to see overfitting continue to worsen
degrees_to_test = list(range(1, 16))
train_mses = []
test_mses = []

for deg in degrees_to_test:
    _, train_mse, test_mse = fit_and_evaluate(X_train, y_train, X_test, y_test, deg)
    train_mses.append(train_mse)
    test_mses.append(test_mse)

# Create two-panel figure: full scale + zoomed view
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left panel: Full scale (shows dramatic test MSE explosion)
ax1 = axes[0]
ax1.plot(
    degrees_to_test,
    train_mses,
    "o-",
    label="Train MSE",
    color="steelblue",
    linewidth=2,
    markersize=7,
)
ax1.plot(
    degrees_to_test,
    test_mses,
    "o-",
    label="Test MSE",
    color="coral",
    linewidth=2,
    markersize=7,
)

best_deg = degrees_to_test[np.argmin(test_mses)]
best_test_mse = min(test_mses)
ax1.axvline(
    x=best_deg,
    color="green",
    linestyle="--",
    alpha=0.5,
    label=f"Best degree = {best_deg}",
)
ax1.scatter([best_deg], [best_test_mse], color="green", s=150, zorder=5, marker="*")

ax1.set_xlabel("Polynomial Degree", fontsize=11)
ax1.set_ylabel("MSE", fontsize=11)
ax1.set_title("Full Scale: Test MSE Explodes!", fontsize=12)
ax1.set_xticks(degrees_to_test)
ax1.legend(loc="upper left")
ax1.annotate(
    "Overfitting\nhas no limit!",
    xy=(15, test_mses[-1]),
    xytext=(12, test_mses[-1] * 0.7),
    fontsize=10,
    ha="center",
    color="red",
    arrowprops=dict(arrowstyle="->", color="red", alpha=0.7),
)

# Right panel: Zoomed view (shows training MSE decreasing)
ax2 = axes[1]
ax2.plot(
    degrees_to_test,
    train_mses,
    "o-",
    label="Train MSE",
    color="steelblue",
    linewidth=2,
    markersize=7,
)
ax2.plot(
    degrees_to_test,
    test_mses,
    "o-",
    label="Test MSE",
    color="coral",
    linewidth=2,
    markersize=7,
)

ax2.axvline(
    x=best_deg,
    color="green",
    linestyle="--",
    alpha=0.5,
    label=f"Best degree = {best_deg}",
)
ax2.scatter([best_deg], [best_test_mse], color="green", s=150, zorder=5, marker="*")

# Zoom to show training MSE detail
ax2.set_ylim(0, 15)  # Zoomed scale
ax2.set_xlabel("Polynomial Degree", fontsize=11)
ax2.set_ylabel("MSE", fontsize=11)
ax2.set_title("Zoomed: Training MSE Keeps Dropping", fontsize=12)
ax2.set_xticks(degrees_to_test)
ax2.legend(loc="upper right")
ax2.annotate(
    "Train MSE\n↓ keeps falling",
    xy=(10, train_mses[9]),
    xytext=(7, 6),
    fontsize=10,
    ha="center",
    color="steelblue",
    arrowprops=dict(arrowstyle="->", color="steelblue", alpha=0.7),
)
ax2.annotate(
    "Underfitting",
    xy=(1, train_mses[0]),
    xytext=(2.5, train_mses[0] + 2),
    fontsize=10,
    ha="center",
    color="gray",
)

plt.suptitle(
    f"The Classic Overfitting Curve (True function: degree 2, Noise σ = {noise_std})",
    fontsize=14,
    y=1.02,
)
plt.tight_layout()
plt.show()

print("📈 Train MSE keeps decreasing — the model fits training data better and better.")
print("📉 Test MSE keeps INCREASING — overfitting has no limit!")
print(f"⭐ The sweet spot is degree {best_deg} — matching the true function.")
print(
    f"\n🔥 At degree 15: Train MSE = {train_mses[-1]:.2f}, Test MSE = {test_mses[-1]:.1f}"
)
print(
    f"   That's {test_mses[-1] / best_test_mse:.0f}x worse than optimal on test data!"
)

In [ ]:
# What does degree 15 actually LOOK like? Let's see the wiggly madness!
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
X_smooth = np.linspace(-3, 3, 300).reshape(-1, 1)

showcase_degrees = [2, 9, 15]
colors = ["green", "orange", "red"]
titles = ["Degree 2 (Correct)", "Degree 9 (Overfit)", "Degree 15 (Extreme Overfit)"]

for ax, deg, color, title in zip(axes, showcase_degrees, colors, titles):
    model, train_mse, test_mse = fit_and_evaluate(X_train, y_train, X_test, y_test, deg)

    sns.scatterplot(
        x=X_train.flatten(), y=y_train, s=70, ax=ax, label="Train", alpha=0.7
    )
    sns.scatterplot(
        x=X_test.flatten(), y=y_test, s=70, marker="s", ax=ax, label="Test", alpha=0.7
    )

    # Plot the fitted curve
    y_pred_smooth = model.predict(X_smooth)
    ax.plot(
        X_smooth.flatten(),
        y_pred_smooth,
        color=color,
        linewidth=2,
        label=f"Fit (deg {deg})",
    )

    # True function
    ax.plot(
        X_smooth.flatten(),
        np.poly1d(TRUE_POLY_COEFFS[2])(X_smooth.flatten()),
        "k--",
        alpha=0.5,
        label="True function",
    )

    ax.set_title(f"{title}\nTrain MSE: {train_mse:.2f} | Test MSE: {test_mse:.2f}")
    ax.set_xlabel("X")
    ax.set_ylabel("y")
    ax.set_ylim(-10, 20)
    ax.legend(fontsize=8)

plt.suptitle("What Overfitting Looks Like: The Wiggly Madness", y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

print(
    "👀 As degree increases, the polynomial contorts itself to hit every training point."
)
print("   But this 'flexibility' is exactly what destroys generalization!")

### 🎓 Bonus: Runge's Phenomenon

_Optional reading for curious students!_

The wild oscillations you see at the edges of the degree-15 fit have a name: **Runge's phenomenon**, discovered by German mathematician Carl Runge in 1901.

**What's happening:** When fitting high-degree polynomials to evenly-spaced points, the polynomial tends to oscillate wildly near the boundaries — even if the underlying function is perfectly smooth! The more points and higher the degree, the worse the oscillations become at the edges.

**Why it matters:** This isn't just a quirk of polynomials. It's an early example of a general principle: **more flexibility isn't always better**. A model with too many parameters will find ways to fit the training data that have nothing to do with the true underlying pattern.

This is exactly the overfitting problem we've been studying — Runge just discovered it 100+ years before "machine learning" existed!


## 13. Key Takeaways

### What We Learned

1. **Polynomial regression with sklearn** uses `PolynomialFeatures` + `LinearRegression` in a pipeline
   - The `fit()` → `predict()` pattern is consistent across all sklearn models

2. **Mean Squared Error (MSE)** measures prediction quality
   - Lower is better — but only when measured correctly!

3. **Train/test split** is essential for honest evaluation
   - Training error alone is misleading
   - Test error estimates real-world performance

4. **Overfitting** = model too complex for the data
   - Symptom: Low train error, high test error (the "gap")
   - Cause: Model memorizes noise instead of learning the pattern
   - There's no limit — degree 15 was 60x worse than optimal!

5. **The classic U-shaped curve** appears everywhere in ML
   - Too simple → underfitting (high train AND test error)
   - Just right → sweet spot (low test error)
   - Too complex → overfitting (low train, high test error)


### Reflection Questions

Think about and discuss with your classmates:

1. **Why does a high-degree polynomial have lower training error even on simple data?**

2. **Looking at the degree-15 visualization — why does the polynomial go crazy at the edges?**

3. **If you only had training data (no test set), how might you be fooled by an overfit model?**

4. **In real ML problems, you don't know the "true" underlying function. How would you choose the right model complexity?**

5. **Neural networks can have millions of parameters. How do you think they avoid overfitting?**


### The Bottom Line

> **Never trust training error alone. Always evaluate on held-out test data.**

> **Be skeptical of models that fit training data "too well" — they might be fitting noise.**

> **Simpler models often generalize better (Occam's Razor).**
